In [1]:
import pandas as pd
from sklearn.impute import KNNImputer 
from sklearn.preprocessing import MinMaxScaler


import numpy as np

from imblearn.under_sampling import RandomUnderSampler

from sklearn import preprocessing, model_selection, pipeline

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import LabelEncoder

In [9]:
# df = df[['TARGET', 'CODE_GENDER', 'CNT_CHILDREN', 'FLAG_OWN_CAR','FLAG_OWN_REALTY',
#          'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'DAYS_BIRTH', 'AMT_ANNUITY']]

In [4]:
#df1 = pd.read_csv("application_train.csv")
# df1.columns[df1.isna().mean() < .5]

In [2]:
data = pd.read_csv("application_train.csv")

In [3]:
data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
data = data.drop(columns=["SK_ID_CURR"])

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 121 entries, TARGET to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(40), object(16)
memory usage: 283.9+ MB


In [12]:
data.columns[data.dtypes.values =='int64'].shape

(40,)

In [14]:
data.columns[data.dtypes.values =='float64'].shape

(65,)

In [16]:
data.columns[data.dtypes.values =='int64'].union(data.columns[data.dtypes.values =='float64'])

Index(['AMT_ANNUITY', 'AMT_CREDIT', 'AMT_GOODS_PRICE', 'AMT_INCOME_TOTAL',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_YEAR',
       ...
       'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'TARGET',
       'TOTALAREA_MODE', 'YEARS_BEGINEXPLUATATION_AVG',
       'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BEGINEXPLUATATION_MODE',
       'YEARS_BUILD_AVG', 'YEARS_BUILD_MEDI', 'YEARS_BUILD_MODE'],
      dtype='object', length=105)

# PREPROCESSING

## TRAITEMENT DES VALEURS NAN

In [33]:
data.isna().mean()

TARGET                        0.000000
NAME_CONTRACT_TYPE            0.000000
CODE_GENDER                   0.000000
FLAG_OWN_CAR                  0.000000
FLAG_OWN_REALTY               0.000000
                                ...   
AMT_REQ_CREDIT_BUREAU_DAY     0.135016
AMT_REQ_CREDIT_BUREAU_WEEK    0.135016
AMT_REQ_CREDIT_BUREAU_MON     0.135016
AMT_REQ_CREDIT_BUREAU_QRT     0.135016
AMT_REQ_CREDIT_BUREAU_YEAR    0.135016
Length: 80, dtype: float64

In [34]:
data.isna().mean().min(), data.isna().mean().max()

(0.0, 0.4976082156410665)

In [17]:
data = data[data.columns[data.isna().mean() < .5]]

In [18]:
data.shape

(307511, 80)

In [11]:
data.dtypes.unique()
# df[df.columns[df.isna().mean() < 0]]

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

### IMPUTATION DES DONNEES NUMERIQUES PAR LA MEDIANE, ET DES DONNEES CATEGORIELLE PAR LE MODE

In [23]:
data.columns

Index(['TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE',
       'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'FLAG_MOBIL', 'FLAG_EMP_PHONE',
       'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL',
       'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START',
       'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION',
       'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION',
       'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY',
       'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'EXT_SOURCE_2',
       'EXT_SOURCE_3', 'YEARS_BEGINEXPLUATATION_AVG', 'FLOORSMAX_AVG',
       'YE

In [24]:
data.shape

(307511, 80)

In [19]:
df = data.copy()

In [34]:
# data.columns[data.dtypes.values =='object']

In [24]:
#data[data.columns[data.dtypes.values =='int64'].union(data.columns[data.dtypes.values =='float64'])]

In [23]:
df = pd.concat([pd.DataFrame(SimpleImputer(strategy='median').fit_transform(data[data.columns[data.dtypes.values =='int64'].union(data.columns[data.dtypes.values =='float64'])]),
                             columns=data.columns[data.dtypes.values =='int64'].union(data.columns[data.dtypes.values =='float64'])),
          pd.DataFrame(SimpleImputer(strategy='most_frequent').fit_transform(data[data.columns[data.dtypes.values =='object']]),
                       columns=data.columns[data.dtypes.values =='object'])],
               axis=1)

In [26]:
df.shape

(307511, 80)

In [27]:
df.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,EMERGENCYSTATE_MODE
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,No
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,N,Family,State servant,Higher education,Married,House / apartment,Core staff,MONDAY,School,No
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,MONDAY,Government,No
3,29686.5,312682.5,297000.0,135000.0,0.0,0.0,0.0,0.0,0.0,1.0,...,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,No
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,THURSDAY,Religion,No


## TRAITEMENT DES VARIABLES CATEGORIELLES

In [28]:
for col in df.select_dtypes('object'):
    print("################### ", col, "##################\n", df[col].value_counts(), "\n")

###################  NAME_CONTRACT_TYPE ##################
 Cash loans         278232
Revolving loans     29279
Name: NAME_CONTRACT_TYPE, dtype: int64 

###################  CODE_GENDER ##################
 F      202448
M      105059
XNA         4
Name: CODE_GENDER, dtype: int64 

###################  FLAG_OWN_CAR ##################
 N    202924
Y    104587
Name: FLAG_OWN_CAR, dtype: int64 

###################  FLAG_OWN_REALTY ##################
 Y    213312
N     94199
Name: FLAG_OWN_REALTY, dtype: int64 

###################  NAME_TYPE_SUITE ##################
 Unaccompanied      249818
Family              40149
Spouse, partner     11370
Children             3267
Other_B              1770
Other_A               866
Group of people       271
Name: NAME_TYPE_SUITE, dtype: int64 

###################  NAME_INCOME_TYPE ##################
 Working                 158774
Commercial associate     71617
Pensioner                55362
State servant            21703
Unemployed                 

In [29]:
# supprimons les lignes contenant la valeur "Unknown" pour la colonne "NAME_FAMILY_STATUS"
df.drop(df.loc[df["NAME_FAMILY_STATUS"] == "Unknown"].index, inplace=True)

In [30]:
df["NAME_INCOME_TYPE"] = df["NAME_INCOME_TYPE"].replace(["Unemployed", "Student", "Businessman", "Maternity leave"], "Autres")

In [31]:
df["NAME_EDUCATION_TYPE"] = df["NAME_EDUCATION_TYPE"].replace(["Secondary / secondary special", "Lower secondary"], "secondary")\
                                                        .replace(["Higher education", "Incomplete higher"], "higher")

In [32]:
df = df.drop(columns=["ORGANIZATION_TYPE", "WEEKDAY_APPR_PROCESS_START", "OCCUPATION_TYPE", "NAME_TYPE_SUITE"])

### ENCODAGES DES VARIABLES CATEGORIELLES

In [21]:
# Pour l'encodage des variables catégorielles, notre choix s'est porté sur LabelEncoder
lblEnc = LabelEncoder()
for col in df.select_dtypes('object').columns:
    df[col] = lblEnc.fit_transform(df[col].astype(str))

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 307509 entries, 0 to 307510
Data columns (total 36 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   AMT_INCOME_TOTAL              307509 non-null  float64
 1   AMT_CREDIT                    307509 non-null  float64
 2   AMT_ANNUITY                   307509 non-null  float64
 3   AMT_GOODS_PRICE               307509 non-null  float64
 4   REGION_POPULATION_RELATIVE    307509 non-null  float64
 5   DAYS_REGISTRATION             307509 non-null  float64
 6   CNT_FAM_MEMBERS               307509 non-null  float64
 7   EXT_SOURCE_2                  307509 non-null  float64
 8   EXT_SOURCE_3                  307509 non-null  float64
 9   YEARS_BEGINEXPLUATATION_AVG   307509 non-null  float64
 10  FLOORSMAX_AVG                 307509 non-null  float64
 11  YEARS_BEGINEXPLUATATION_MODE  307509 non-null  float64
 12  FLOORSMAX_MODE                307509 non-nul

In [42]:
pd.concat([df, data.TARGET],1).to_csv('data_cleaned.csv')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  """Entry point for launching an IPython kernel.


In [22]:
df.to_csv('data_cleaned.csv')

## Traitement des Valeur Nan par KNNImputer

In [ ]:
df = pd.read_csv("application_train.csv")
df = df.drop(columns=["SK_ID_CURR"])
df = df[df.columns[df.isna().mean() < .5]]

imputer = KNNImputer()
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
df.to_csv("data_cleaned_kknimp.csv")
df.head()

# FEATURES INGENEERING

In [8]:
df = pd.read_csv('application_train.csv')
df = df[['TARGET', 'CODE_GENDER', 'CNT_CHILDREN', 'FLAG_OWN_CAR','FLAG_OWN_REALTY',
         'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'DAYS_BIRTH', 'AMT_ANNUITY',]]
df = df.dropna().reset_index(drop=True)
df.head()

,TARGET,CODE_GENDER,CNT_CHILDREN,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,DAYS_BIRTH,AMT_ANNUITY
0,1,M,0,N,Y,202500.0,406597.5,-9461,24700.5
1,0,F,0,N,N,270000.0,1293502.5,-16765,35698.5
2,0,M,0,Y,Y,67500.0,135000.0,-19046,6750.0
3,0,F,0,N,Y,135000.0,312682.5,-19005,29686.5
4,0,M,0,N,Y,121500.0,513000.0,-19932,21865.5


In [9]:
df.AMT_INCOME_TOTAL.min(), df.AMT_INCOME_TOTAL.max(), df.AMT_INCOME_TOTAL.mean()

(25650.0, 117000000.0, 168797.23335989713)

In [10]:
# creation d'une variable calculée à partir du montant total
df["class_amt_tot"] = df.AMT_INCOME_TOTAL.apply(lambda x : "low" if x < 168796 else ("meduim" if (x >= 168796 or x < 1500000) else "hight"))

In [11]:
df.AMT_CREDIT.min(), df.AMT_CREDIT.max(), df.AMT_CREDIT.mean()

(45000.0, 4050000.0, 599027.2933489215)

In [12]:
# creation d'une variable calculée à partir du montant du credit
df["class_amt_credit"] = df.AMT_CREDIT.apply(lambda x : "low" if x < 599029 else ("meduim" if (x >= 599029 or x < 1000000) else "hight"))

In [13]:
df.AMT_ANNUITY.min(), df.AMT_ANNUITY.max(), df.AMT_ANNUITY.mean()

(1615.5, 258025.5, 27108.573909183444)

In [14]:
# creation d'une variable calculée à partir du montant DE LA RENTE
df["class_amt_ANNUITY"] = df.AMT_ANNUITY.apply(lambda x : "low" if x < 27108 else ("meduim" if (x >= 27108 or x < 100000) else "hight"))


In [15]:
df.DAYS_BIRTH.min(), df.DAYS_BIRTH.max(), df.DAYS_BIRTH.mean()

(-25229, -7489, -16037.054211558412)

In [16]:
df.DAYS_BIRTH = abs(df.DAYS_BIRTH)
df["class_age"] = df.DAYS_BIRTH.apply(lambda x : "jeune" if x < 10000 else ("adulte" if x >= 10000 or x < 20000 else "vieux"))

In [18]:
df.CNT_CHILDREN.value_counts()

0     215359
1      61119
2      26749
3       3717
4        429
5         84
6         21
7          7
14         3
8          2
9          2
12         2
10         2
19         2
11         1
Name: CNT_CHILDREN, dtype: int64

In [20]:
df["CNT_CHILDREN_class"] = df.CNT_CHILDREN.apply(lambda x : "3 et plus" if x >= 3 else str(x))

In [21]:
df.head()

,TARGET,CODE_GENDER,CNT_CHILDREN,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,DAYS_BIRTH,AMT_ANNUITY,class_amt_tot,class_amt_credit,class_amt_ANNUITY,class_age,CNT_CHILDREN_class
0,1,M,0,N,Y,202500.0,406597.5,9461,24700.5,meduim,low,low,jeune,0
1,0,F,0,N,N,270000.0,1293502.5,16765,35698.5,meduim,meduim,meduim,adulte,0
2,0,M,0,Y,Y,67500.0,135000.0,19046,6750.0,low,low,low,adulte,0
3,0,F,0,N,Y,135000.0,312682.5,19005,29686.5,low,low,meduim,adulte,0
4,0,M,0,N,Y,121500.0,513000.0,19932,21865.5,low,low,low,adulte,0


In [24]:
df.CODE_GENDER.value_counts()

F      202440
M      105055
XNA         4
Name: CODE_GENDER, dtype: int64

In [29]:
df = df.drop(df[df.CODE_GENDER == "XNA"].index).reset_index(drop=True)

In [30]:
df.to_csv('final_data_cleanned.csv')